### 药企圈OpenAPI接口测试

#### 一、环境选择

In [1]:
# coding=utf-8
import pandas as pd
from colorama import Fore,Back,Style
from api import servers,interfaces
from tester import testers

DISPLAY_CHOICE = True

if DISPLAY_CHOICE:
    df = pd.DataFrame(servers,index=[0])
    df = df.T
    df.columns=['url']
    print(Fore.GREEN+'----------------------------------------\n',df)
    df = pd.DataFrame(testers)
    df = df.T
    print(Fore.GREEN+'----------------------------------------\n',df)

----------------------------------------
                          url
pre    pre.mediportal.com.cn
pro     xg.mediportal.com.cn
test  test.mediportal.com.cn
----------------------------------------
                          companyId envcode password         user
tester01  5799a1a7b522255a923b4716    test   123456  12157573047
tester02                              test                      


In [2]:
# --修改下面的名称来选择环境-----------------
envcode = 'test'
# --修改下面的名称来选择账号-----------------
testercode = 'tester01'
# ---------------------------------------

In [3]:
server = servers[envcode]
print(Fore.BLUE+'----------------------------------------\n当前选择的环境是: %s --> %s' % (envcode,server))
tester = testers[testercode]
if tester['envcode'] !=envcode: 
    print(Fore.BLUE+'----------------------------------------\n当前选择的账号的环境不匹配: %s --> envcode:%s' % (testercode,tester['envcode']))
else:
    from colorama import Fore
    print(Fore.BLUE+'----------------------------------------\n当前选择的账号是: %s' % testercode)

----------------------------------------
当前选择的环境是: test --> test.mediportal.com.cn
----------------------------------------
当前选择的账号是: tester01


#### 二、接口选择

In [4]:
if DISPLAY_CHOICE:
    print(Fore.GREEN+'----------------------------------------\n',pd.DataFrame([{'可选择的接口类别':x} for x in interfaces.keys()]))

----------------------------------------
             可选择的接口类别
0      basedata_read
1  organization_read
2           crm_read
3          crm_write
4          community


In [5]:
# --修改下面的内容来选择接口类别--------------
interface_catagory = interfaces['crm_read']
# ---------------------------------------

In [6]:
interface_content = interface_catagory['interface']
if DISPLAY_CHOICE:
    print(Fore.GREEN+'----------------------------------------\n',pd.DataFrame([{'可选择的接口':x} for x in interface_catagory['interface']]))

----------------------------------------
                        可选择的接口
0           queryHospitalData
1    queryCompanyHospitalData
2    queryCompanyCustomerData
3        queryFriendsByOpenId
4    queryFriendsByCustomerId
5  queryCustomerLastLoginTime


In [7]:
# --修改下面的内容来选择接口-----------------
interface = interface_content['queryHospitalData']
# ---------------------------------------

In [8]:
api_key = interface_catagory['api_key'][envcode]
api_secret = interface_catagory['api_secret'][envcode]

print(Fore.BLUE+'当前选择的接口是: [%s]\nurl = %s\nmethod = %s\nbody = %s \noutput = %s \nkey = %s\nsecret = %s' 
      % (interface['desc'],interface['url'],interface['method'],
         interface.get('body',False),interface.get('output',[]),
         api_key,api_secret))
pd.DataFrame(interface['params'],columns=['参数']).head(100)

当前选择的接口是: [查询药企圈医院数据]
url = /open-api/drugorg/crm/open/queryHospitalData
method = GET
body = False 
output = ['data', 'pageData'] 
key = 3b00bfb4c77b41e9b7d6eec2180855c6
secret = f88c8e674995445889613eb9b47fc940fxr7k710


,参数
0,appKey
1,nonce
2,time
3,sign
4,timeStamp
5,pageIndex
6,pageSize


#### 三、输入数据准备

In [9]:
# 以下统一的参数无须配置:
# 1.appkey  2.nonce  3.time  4.sign
#
# --修改下面的内容来修改其他输入参数---------
params = {
    'companyId':tester['companyId'],
    'timeStamp':0,
    'pageIndex':1,
    'pageSize':100
}
# --修改下面的内容来修HTTP BODY参数--------
data = {
    
}

In [10]:

def generate_nonce(len = 16, lowercase = True, uppercase = False, digits = True) :
    import random
    import string
    random.seed()
    chars = ''
    if lowercase: chars += string.ascii_lowercase
    if uppercase: chars += string.ascii_uppercase
    if digits: chars += string.digits
    return ''.join([random.choice(chars) for _ in range(len)])

def generate_time() :
    import time
    return int(time.time()*1000)

def generate_sign(params,secret):
    params_list = [key + '=' + str(value) + '&' for key, value in params.items() if value is not None or str(value) != '' ]
    params_list.sort()
    params_str = ''.join(params_list)
    params_str = params_str + 'key=' + secret
    params_str = md5(params_str.encode('utf-8'))
    return params_str
    
def md5(str):
    import hashlib
    m = hashlib.md5()   
    m.update(str)
    return m.hexdigest()

params['appKey'] = api_key
params['nonce'] = generate_nonce()
params['time'] = generate_time()
params['sign'] = generate_sign(params,api_secret)

df=pd.DataFrame(params,index=[0]).T
df.columns=['参数值']
df.head(100)

,参数值
appKey,3b00bfb4c77b41e9b7d6eec2180855c6
companyId,5799a1a7b522255a923b4716
nonce,npqupn7n9uigbg99
pageIndex,1
pageSize,100
sign,16a19cbff2743867cd8a982d84828d44
time,1527520862246
timeStamp,0


#### 四、执行接口

In [11]:
import requests
if interface['method'] == 'GET':
    r = requests.get('http://' + server + interface['url'],params=params)
    print(Fore.BLUE+'[%s请求执行完成] --> %s' % (interface['method'],r.url))
elif interface['method'] == 'POST':
    if interface.get('body',False):
        r = requests.post('http://' + server + interface['url'],params=params,data=data)
    else:
        r = requests.post('http://' + server + interface['url'],params=params)
    print(Fore.BLUE+'[%s请求执行完成] --> %s' % (interface['method'],r.url))
else:
    print(Fore.BLUE+'[%s接口方法填写错误，请求未执行]' % interface['method'])

[GET请求执行完成] --> http://test.mediportal.com.cn/open-api/drugorg/crm/open/queryHospitalData?companyId=5799a1a7b522255a923b4716&timeStamp=0&pageIndex=1&pageSize=100&appKey=3b00bfb4c77b41e9b7d6eec2180855c6&nonce=npqupn7n9uigbg99&time=1527520862246&sign=16a19cbff2743867cd8a982d84828d44


#### 五、输出数据查询

In [12]:
import json
output = json.loads(r.text)
resultCode = output['resultCode']
if resultCode == 1:
    print(Fore.BLUE+'[请求成功]')
    for field in interface['output']:
        output = output[field]
    df=pd.DataFrame(output)
else:
    print(Fore.BLUE+output['请求失败：%s' % output['resultMsg']])
    pass
print(Fore.BLUE+'最前5条数据：')
df.head(5)

[请求成功]
最前5条数据：


,address,city,code,country,id,name,province,status
0,辽宁省抚顺市新抚区站前街道社区卫生服务中心,210400,201102220079,210402,5af67fc404dd3c74c65dfe7a,抚顺市新抚区站前街道社区卫生服务中心,210000,1
1,浙江省杭州市下城区中山北区99号,330100,201103030007,330103,5af67fc404dd3c74c65dfe7b,杭州张同泰中医门诊部有限公司（杭州张同泰药业有限公司）,330000,1
2,浙江省嘉兴市南湖区景湖路,330400,201103070019,330402,5af67fc404dd3c74c65dfe7c,嘉兴东塔门诊部（嘉兴东塔路社区卫生服务站；东塔卫生院）,330000,1
3,辽宁省沈阳市和平区哈尔滨路60号,210100,201103100003,210102,5af67fc404dd3c74c65dfe7d,沈阳市和平区北市社区卫生服务中心（沈阳市和平区惠民医院）,210000,1
4,瓯海区新桥街道金蟾大道119号,330300,201104080021,330304,5af67fc404dd3c74c65dfe7e,温州市瓯海区第二人民医院(温州市瓯海区新桥街道社区卫生服务中心),330000,1


In [13]:
if resultCode == 1:
    pass
print(Fore.BLUE+'最后5条数据：')
df.tail(5)

最后5条数据：


,address,city,code,country,id,name,province,status
95,NaN,130400,201403290005,130423,5af67fc404dd3c74c65dfed9,临漳县中医院,130000,1
96,NaN,130100,201403290026,0,5af67fc404dd3c74c65dfeda,石家庄新乐市妇产医院,130000,1
97,NaN,371500,201403290050,371524,5af67fc404dd3c74c65dfedb,东阿县中医院,370000,1
98,NaN,410400,201403290064,0,5af67fc404dd3c74c65dfedc,峡县人民医院,410000,1
99,NaN,410500,201403290081,410581,5af67fc404dd3c74c65dfedd,林州市中医院,410000,1
